# LangGraph 101 Typescript

[LLMs](https://js.langchain.com/docs/modules/models/chat/) make it possible to embed intelligence into a new class of applications. [LangGraph](https://langchain-ai.github.io/langgraphjs/) is a TypeScript/JavaScript framework to help build applications with LLMs. Here, we will overview the basics of LangGraph, explain its benefits, show how to use it to build workflows and agents, and show how it works with [LangChain JS](https://js.langchain.com/) and [LangSmith](https://smith.langchain.com/).

![ecosystem](img/ecosystem.png)

## Chat models

[Chat models](https://js.langchain.com/docs/modules/models/chat/) are the foundation of LLM applications. In TypeScript, chat models are accessed via standardized interfaces that take an array of message objects and return a response message. LangChain JS provides [a unified interface for chat models](https://js.langchain.com/docs/modules/models/chat/), making it easy to [access many different providers](https://js.langchain.com/docs/integrations/providers/).

In [1]:
// LLM Initialization
// Set your OpenAI API key in your environment or .env file
const OPENAI_API_KEY = process.env.OPENAI_API_KEY;
if (!OPENAI_API_KEY) {
  throw new Error("OPENAI_API_KEY is not set in the environment.");
}

In [20]:
import { initChatModel } from "langchain/chat_models/universal";
import "@langchain/langgraph/zod";
// Initialize the chat model (OpenAI GPT-4.1)
const llm = initChatModel("openai:gpt-4.1", { temperature: 0.0 });

## Running the model

The `initChatModel` interface in LangChain JS provides [standardized methods](https://js.langchain.com/docs/concepts/runnables/) for using chat models, which include:
- `invoke()`: Synchronously process inputs and return outputs
- `stream()`: Return outputs [incrementally](https://js.langchain.com/docs/concepts/streaming/) as they're generated

In [22]:
// Run the model
const result = await llm.stream("What is LangGraph?");

2:26 - Property 'stream' does not exist on type 'Promise<ConfigurableModel<BaseLanguageModelInput, ConfigurableChatModelCallOptions>>'.


In [23]:
// Check the result type
console.log("Result type:", result.type); // Should be "ai" for AIMessage

2:29 - Cannot find name 'result'.


In [5]:
// Print the result content
console.log(result);

AIMessage(content='**LangGraph** is an open-source framework designed for building stateful, multi-agent applications powered by Large Language Models (LLMs). It extends the popular [LangChain](https://www.langchain.com/) ecosystem, focusing on enabling complex workflows where multiple agents (LLMs, tools, or humans) interact in a graph-like structure.\n\n### Key Features\n\n- **Graph-based Workflows:**  \n  LangGraph lets you define workflows as graphs, where each node can be an agent, a tool, or a function, and edges represent possible transitions based on state or output.\n- **Stateful Interactions:**  \n  Unlike simple chains, LangGraph maintains and updates a shared state as the workflow progresses, allowing for memory, context, and iterative reasoning.\n- **Multi-Agent Collaboration:**  \n  Easily orchestrate multiple agents that can communicate, collaborate, or debate to solve complex tasks.\n- **Flexible Control Flow:**  \n  Supports loops, branching, and conditional logic, mak

In [6]:
// Extract the result content
console.log(result.content);

'**LangGraph** is an open-source framework designed for building stateful, multi-agent applications powered by Large Language Models (LLMs). It extends the popular [LangChain](https://www.langchain.com/) ecosystem, focusing on enabling complex workflows where multiple agents (LLMs, tools, or humans) interact in a graph-like structure.\n\n### Key Features\n\n- **Graph-based Workflows:**  \n  LangGraph lets you define workflows as graphs, where each node can be an agent, a tool, or a function, and edges represent possible transitions based on state or output.\n- **Stateful Interactions:**  \n  Unlike simple chains, LangGraph maintains and updates a shared state as the workflow progresses, allowing for memory, context, and iterative reasoning.\n- **Multi-Agent Collaboration:**  \n  Easily orchestrate multiple agents that can communicate, collaborate, or debate to solve complex tasks.\n- **Flexible Control Flow:**  \n  Supports loops, branching, and conditional logic, making it suitable fo

## Tools

[Tools](https://js.langchain.com/docs/concepts/tools/) are utilities that can be called by a chat model. In LangChain JS, you create tools using the `tool` function from `@langchain/core/tools`, often with Zod schemas for runtime validation. This approach automatically infers the tool's name, description, and expected arguments from the function definition. You can also use [Model Context Protocol (MCP) servers](https://github.com/langchain-ai/langchain-mcp-adapters) as LangChain-compatible tools in TypeScript.

In [ ]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

// Define the write_email tool using LangChain's tool function and Zod for validation
 const writeEmail = tool({
  name: "write_email",
  description: "Write and send an email.",
  schema: z.object({
    to: z.string(),
    subject: z.string(),
    content: z.string(),
  }),
  func: async ({ to, subject, content }) => {
    // Placeholder response - in real app would send email
    return `Email sent to ${to} with subject '${subject}' and content: ${content}`;
  },
});

5:21 - Expected 2 arguments, but got 1.


In [5]:
// Check the tool type
console.log("Tool type:", typeof writeEmail); // Should be "function"

langchain_core.tools.structured.StructuredTool

In [6]:
// Tool argument schema (Zod schema)
console.log("Tool argument schema:", writeEmail.schema);

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [7]:
// Tool description
console.log("Tool description:", writeEmail.description);

'Write and send an email.'

## Tool Calling

Tools can be [called](https://js.langchain.com/docs/concepts/tool-calling/) by LLMs. When a tool is bound to the model, the model can choose to call the tool by returning a structured output with tool arguments. In TypeScript, use the `bindTools` method to augment an LLM with tools.

![tool-img](img/tool_call_detail.png)

You can use the [`toolChoice` parameter](https://js.langchain.com/docs/how_to/tool_choice/) to enforce tool calling behavior.

In [8]:
// Bind the writeEmail tool to the model, requiring tool use
const modelWithTools = llm.bindTools([writeEmail], { toolChoice: "required" });

// The model can now call tools
const output = await modelWithTools.invoke("Draft a response to my boss about tomorrow's meeting");

2:24 - Cannot find name 'llm'.
2:39 - Cannot find name 'writeEmail'.


In [9]:
// Extract tool call arguments from the output
const toolCallArgs = output.tool_calls?.[0]?.args;
console.log(toolCallArgs);

2:22 - Cannot find name 'output'.


In [10]:
// Call the tool with the extracted arguments
if (toolCallArgs) {
    const toolResult = await writeEmail.func(toolCallArgs);
    console.log(toolResult); // "Email sent to boss@company.com drafted with subject ..."
  }

2:5 - Cannot find name 'toolCallArgs'.
3:30 - Cannot find name 'writeEmail'.
3:46 - Cannot find name 'toolCallArgs'.


Above, we enforce tool calling by setting `toolChoice: "required"`, so the model will always call a tool to write an email.

![basic_prompt](img/tool_call.png)

## Workflows

There are many patterns for building applications with LLMs.

[You can embed LLM calls into pre-defined workflows](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/), giving the system more agency to make decisions.

As an example, you could add a router step to determine whether to write an email or not.

![workflow_example](img/workflow_example.png)

## Agents

You can further increase agency, allowing the LLM to dynamically direct its own tool usage.

[Agents](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/) are typically implemented as tool calling in a loop, where the output of each tool call is used to inform the next action.

![agent_example](img/agent_example.png)

Agents are well suited to open-ended problems where it's difficult to predict the *exact* steps needed in advance.

Workflows are often appropriate when the control flow can easily be defined in advance.

![workflow_v_agent](img/workflow_v_agent.png)

## What is LangGraph?

[LangGraph](https://langchain-ai.github.io/langgraphjs/concepts/high_level/) provides low-level supporting infrastructure that sits underneath *any* workflow or agent.

It does not abstract prompts or architecture, and provides a few benefits:

- **Control**: Make it easy to define and/or combine agents and workflows.
- **Persistence**: Provide a way to persist the state of a graph, which enables both memory and human-in-the-loop.
- **Testing, Debugging, and Deployment**: Provide an easy onramp for testing, debugging, and deploying applications.

### Control

LangGraph lets you define your application as a graph with:

1. *State*: What information do we need to track over the course of the application?
2. *Nodes*: How do we want to update this information over the course of the application?
3. *Edges*: How do we want to connect these nodes together?

You can use the [`StateGraph` class](https://langchain-ai.github.io/langgraphjs/concepts/low_level/) to initialize a LangGraph graph with a [Zod schema](https://langchain-ai.github.io/langgraphjs/how-tos/define-state/) for your state.

`State` defines the schema for information you want to track over the course of the application.

In TypeScript, this is typically a Zod object schema, which provides runtime validation and type safety.

In [11]:
import { StateGraph, START, END } from "@langchain/langgraph";
import { z } from "zod";

// Define the state schema using Zod
const StateSchema = z.object({
  request: z.string(),
  email: z.string(),
});

// Initialize the workflow graph
const workflow = new StateGraph<StateSchemaType>(StateSchema);

11:33 - Cannot find name 'StateSchemaType'.


Each node is simply a TypeScript function. This gives you full control over the logic inside each node.

Nodes receive the current state and return an object to update the state.

By default, [state keys are overwritten](https://langchain-ai.github.io/langgraphjs/how-tos/state-reducers/).

However, you can [define custom update logic (reducers)](https://langchain-ai.github.io/langgraphjs/concepts/low_level/#reducers) to control how state is merged or updated.

![nodes_edges](img/nodes_edges.png)

In [12]:
// Node function for writing an email
const writeEmailNode = async (state) => {
    // Imperative code that processes the request
    const output = await modelWithTools.invoke(state.request);
    const args = output.tool_calls?.[0]?.args;
    let email = "";
    if (args) {
      email = await writeEmail.func(args);
    }
    return { email };
  };

4:26 - Cannot find name 'modelWithTools'.
8:21 - Cannot find name 'writeEmail'.


Edges connect nodes together. 

We specify the control flow by adding edges and nodes to our state graph. 

In [13]:
// Add the node and edges to the workflow
workflow
  .addNode("write_email_node", writeEmailNode)
  .addEdge(START, "write_email_node")
  .addEdge("write_email_node", END);

// Compile the workflow
const app = workflow.compile();

2:1 - Cannot find name 'workflow'.
3:32 - Cannot find name 'writeEmailNode'.
4:12 - Cannot find name 'START'.
5:32 - Cannot find name 'END'.
8:13 - Cannot find name 'workflow'.


In [25]:
// Run the workflow
const workflowResult = await app.invoke({ request: "Draft a response to my boss about tomorrow's meeting" });
console.log(workflowResult);

{'request': "Draft a response to my boss about tomorrow's meeting",
 'email': "Email sent to boss@example.com with subject 'Re: Tomorrow's Meeting' and content: Hi [Boss's Name],\n\nI hope this message finds you well. I am writing to confirm my attendance at tomorrow's meeting. Please let me know if there are any specific topics or materials you would like me to prepare in advance.\n\nLooking forward to our discussion.\n\nBest regards,\n\n[Your Name]"}

Routing between nodes can be done [conditionally](https://langchain-ai.github.io/langgraphjs/concepts/low_level/#conditional-edges) using a simple function.

The return value of this function is used as the name of the node (or list of nodes) to send the state to next.

You can optionally provide a mapping object that maps the output of your router function to the name of the next node.

In [14]:
import { z } from "zod";
import { StateGraph, START, END } from "@langchain/langgraph";
import { Message, AIMessage, ToolMessage } from "@langchain/langgraph-sdk";

// Define the MessagesState schema
const MessagesState = z.object({
  messages: z.array(z.any()), // Use a more specific type if available
});


// Node: call_llm
const callLlm = async (state) => {
  const output = await modelWithTools.invoke(state.messages);
  return { messages: [output] };
};

// Node: run_tool
const runTool = async (state) => {
  const lastMessage = state.messages[state.messages.length - 1] ;
  const result = [];
  if (lastMessage.tool_calls) {
    for (const toolCall of lastMessage.tool_calls) {
      const observation = await writeEmail.func(toolCall.args);
      result.push({
        type: "tool",
        content: observation,
        tool_call_id: toolCall.id,
      });
    }
  }
  return { messages: result };
};

// Conditional router
const shouldContinue = (state) => {
  const messages = state.messages;
  const lastMessage = messages[messages.length - 1] ;
  if (lastMessage.tool_calls && lastMessage.tool_calls.length > 0) {
    return "run_tool";
  }
  return END;
};

// Build the workflow
const messagesWorkflow = new StateGraph<MessagesStateType>(MessagesState)
  .addNode("call_llm", callLlm)
  .addNode("run_tool", runTool)
  .addEdge(START, "call_llm")
  .addConditionalEdges("call_llm", shouldContinue, { run_tool: "run_tool", [END]: END })
  .addEdge("run_tool", END);

const messagesApp = messagesWorkflow.compile();

3:49 - Cannot find module '@langchain/langgraph-sdk' or its corresponding type declarations.
13:24 - Cannot find name 'modelWithTools'.
23:33 - Cannot find name 'writeEmail'.
45:41 - Cannot find name 'MessagesStateType'.


In [32]:
// Run the workflow (visualization is not included in TypeScript)
// Example invocation:
const messagesResult = await messagesApp.invoke({
    messages: [
      { type: "human", content: "Draft a response to my boss about tomorrow's meeting" }
    ]
  });
  console.log(messagesResult);

[HumanMessage(content="Draft a response to my boss about tomorrow's meeting", additional_kwargs={}, response_metadata={}, id='5e2d9fe3-c075-47ba-a253-f06ffb67aab5'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iIS382COCMtqYtfwIfBBf8P9', 'function': {'arguments': '{"to":"boss@example.com","subject":"Re: Tomorrow\'s Meeting","content":"Hi [Boss\'s Name],\\n\\nI hope this message finds you well. I am writing to confirm my attendance at tomorrow\'s meeting. Please let me know if there are any specific topics or materials you would like me to prepare in advance.\\n\\nLooking forward to our discussion.\\n\\nBest regards,\\n\\n[Your Name]"}', 'name': 'write_email'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 61, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': 

With these low-level components, you can build many different workflows and agents. See the [workflows tutorial](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/) for more examples!

Because agents are such a common pattern, [LangGraph](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/#pre-built) provides a [pre-built agent abstraction](https://langchain-ai.github.io/langgraphjs/agents/overview/).

With LangGraph's [pre-built method](https://langchain-ai.github.io/langgraphjs/how-tos/create-react-agent/), you just pass in the LLM, tools, and prompt.

In [36]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";

// Create a prebuilt React agent
const agent = createReactAgent({
  model: llm,
  tools: [writeEmail],
  prompt: "Respond to the user's request using the tools provided.",
});

// Run the agent
const agentResult = await agent.invoke({
  messages: [{ type: "human", content: "Draft a response to my boss about tomorrow's meeting" }],
});
console.log(agentResult);

{'messages': [HumanMessage(content="Draft a response to my boss about tomorrow's meeting", additional_kwargs={}, response_metadata={}, id='b38b8654-8947-4f76-9d33-a3c30bb1ad1f'),
  AIMessage(content='Of course! Could you please provide a bit more detail? For example:\n\n- What is the purpose or topic of the meeting?\n- Do you want to confirm your attendance, ask to reschedule, or provide any specific information?\n- Is there a particular tone you’d like (formal, friendly, etc.)?\n\nLet me know so I can tailor the response to your needs!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 71, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_cf18407276', 'id': 'chatcmpl-BUJdSwIfBhV1zKy9CFF

### Persistence

It can be very useful to allow agents to pause and gather human feedback.

LangGraph has a built-in persistence layer, implemented through [checkpointers](https://langchain-ai.github.io/langgraphjs/concepts/persistence/#checkpoints), to enable this.

When you compile a graph with a checkpointer, the checkpointer saves a checkpoint of the graph state at every step.

Checkpoints are saved to a thread, which can be accessed after graph execution.

![checkpointer](img/checkpoints.png)

In [41]:
import { InMemorySaver } from "@langchain/langgraph";

// Create a React agent with in-memory checkpointing
const agentWithCheckpoint = createReactAgent({
  model: llm,
  tools: [writeEmail],
  prompt: "Respond to the user's request using the tools provided.",
  checkpointer: new InMemorySaver(),
});

// Config for thread id
const config = { configurable: { thread_id: "1" } };

// Run the agent with checkpointing
const checkpointResult = await agentWithCheckpoint.invoke(
  {
    messages: [{ type: "human", content: "What are some good practices for writing emails?" }],
  },
  config
);
console.log(checkpointResult);

In [43]:
// Get the latest state snapshot for the thread
const state = await agentWithCheckpoint.getState(config);
for (const message of state.values.messages) {
  // You can define a prettyPrint function or just log the message
  console.log(message);
}

================================ Human Message =================================

What are some good practices for writing emails?
================================== Ai Message ==================================

Here are some good practices for writing effective emails:

1. **Use a Clear Subject Line:** Make your subject concise and informative so the recipient knows what to expect.

2. **Greet Appropriately:** Start with a polite greeting, using the recipient’s name if possible.

3. **Be Concise and to the Point:** State your purpose early and keep your message focused.

4. **Use Proper Grammar and Spelling:** Proofread your email to avoid errors and maintain professionalism.

5. **Structure Your Email:** Use short paragraphs, bullet points, or numbered lists for clarity.

6. **Be Polite and Professional:** Use courteous language and avoid slang or overly casual expressions.

7. **Include a Call to Action:** Clearly state what you need from the recipient, if applicable.

8. **Sign Of

In [44]:
// Get the latest state snapshot for the thread
const state = await agentWithCheckpoint.getState(config);
for (const message of state.values.messages) {
  // You can define a prettyPrint function or just log the message
  console.log(message);
}

================================ Human Message =================================

What are some good practices for writing emails?
================================== Ai Message ==================================

Here are some good practices for writing effective emails:

1. **Use a Clear Subject Line:** Make your subject concise and informative so the recipient knows what to expect.

2. **Greet Appropriately:** Start with a polite greeting, using the recipient’s name if possible.

3. **Be Concise and to the Point:** State your purpose early and keep your message focused.

4. **Use Proper Grammar and Spelling:** Proofread your email to avoid errors and maintain professionalism.

5. **Structure Your Email:** Use short paragraphs, bullet points, or numbered lists for clarity.

6. **Be Polite and Professional:** Use courteous language and avoid slang or overly casual expressions.

7. **Include a Call to Action:** Clearly state what you need from the recipient, if applicable.

8. **Sign Of

In [45]:
// Continue the conversation again
const finalResult = await agentWithCheckpoint.invoke(
    {
      messages: [
        { type: "human", content: "I like this, let's write the email" },
      ],
    },
    config
  );
  for (const m of finalResult.messages) {
    console.log(m);
  }

================================ Human Message =================================

What are some good practices for writing emails?
================================== Ai Message ==================================

Here are some good practices for writing effective emails:

1. **Use a Clear Subject Line:** Make your subject concise and informative so the recipient knows what to expect.

2. **Greet Appropriately:** Start with a polite greeting, using the recipient’s name if possible.

3. **Be Concise and to the Point:** State your purpose early and keep your message focused.

4. **Use Proper Grammar and Spelling:** Proofread your email to avoid errors and maintain professionalism.

5. **Structure Your Email:** Use short paragraphs, bullet points, or numbered lists for clarity.

6. **Be Polite and Professional:** Use courteous language and avoid slang or overly casual expressions.

7. **Include a Call to Action:** Clearly state what you need from the recipient, if applicable.

8. **Sign Of

### Testing, Debugging, and Deployment

When using LangChain or LangGraph JS, [LangSmith logging works out of the box](https://docs.smith.langchain.com/observability/how_to_guides/trace_with_langgraph) by setting the following environment variables:


In [ ]:
// Set LangSmith tracing environment variables in your .env or process environment
// Example (do this in your shell or .env file, not in code):
// process.env.LANGSMITH_TRACING = "true";
// process.env.LANGSMITH_API_KEY = "<your-langsmith-api-key>";

Here is the LangSmith trace from above graph execution:

https://smith.langchain.com/public/6f77014f-d054-44ed-aa2c-8b06ceab689f/r

We can see that the agent is able to continue the conversation from the previous state because we used a checkpointer.

It's also easy to deploy to deploy our graph using [LangGraph Platform](https://langchain-ai.github.io/langgraphjs/concepts/langgraph_platform/). 

We simply need to ensure our project has [a structure](https://langchain-ai.github.io/langgraphjs/cloud/deployment/setup_javascript/) like this:

```
my-app/
├── my_agent # all project code lies within here
│   └── agent.py # code for constructing your graph
├── .env # environment variables
├── langgraph.json  # configuration file for LangGraph
└── pyproject.toml # dependencies for your project
```

The `langgraph.json` file specifies the dependencies, graphs, environment variables, and other settings required to deploy a LangGraph application.

To test this, let's deploy `langgraph_101.py`. We have it in our `langgraph.json` file in this repo:

```
 "langgraph101": "./src/email_assistant/langgraph101.py:app",
```

There are a range of [deployment options](https://langchain-ai.github.io/langgraphjs/tutorials/deployment/). 

* All create an API [server](https://langchain-ai.github.io/langgraphjs/concepts/langgraph_server/) for our graph
* All include an interactive IDE (LangGraph [Studio](https://langchain-ai.github.io/langgraphjs)).
 
We can start a deployment locally using `langgraph dev`:

Here we can see a visualization of the graph as well as the graph state in Studio.

![langgraph_studio](img/langgraph_studio.png)